In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("./new_preprocessed_final.csv")

In [5]:
# BERT similarity only
bert_feat = None
for c in df.columns:
  if "bert_similarity" in c.lower() or ("cosine" in c.lower() and "bert" in c.lower()):
    bert_feat = c
    break

if bert_feat is None:
  raise ValueError("BERT similarity feature not found.")

# Label
label_col = None
for c in df.columns:
  if "is_clickbait" in c.lower():
    label_col = c
    break

X = df[[bert_feat]]
y = df[label_col]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42, stratify=y
)

# Perceptron pipeline
pipe = Pipeline([
  ("scaler", StandardScaler()),
  ("clf", Perceptron(max_iter=1000, tol=1e-3, class_weight="balanced", random_state=42))
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print("=== Feature used ===")
print(bert_feat)
print("\n=== Classification Report (SLP + BERT only) ===")
print(classification_report(y_test, y_pred))
print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

=== Feature used ===
bert_similarity

=== Classification Report (SLP + BERT only) ===
              precision    recall  f1-score   support

           0       0.87      0.96      0.91       514
           1       0.38      0.14      0.20        86

    accuracy                           0.84       600
   macro avg       0.62      0.55      0.56       600
weighted avg       0.80      0.84      0.81       600


=== Confusion Matrix ===
[[494  20]
 [ 74  12]]


In [6]:
# BERT similarity + extra features
bert_feat = None
for c in df.columns:
  if "bert_similarity" in c.lower() or ("cosine" in c.lower() and "bert" in c.lower()):
    bert_feat = c
    break

if bert_feat is None:
  raise ValueError("BERT similarity feature not found.")

# Label
label_col = None
for c in df.columns:
  if "is_clickbait" in c.lower():
    label_col = c
    break

# Extra features (linguistic & stylistic)
extra_feats = ["title_length","sensational_word_count","punctuation_count","punctuation_ratio"]
extra_feats = [c for c in extra_feats if c in df.columns]

# Combine BERT + extras
feature_cols = [bert_feat] + extra_feats

# Label column
label_col = next(c for c in df.columns if "is_clickbait" in c.lower())

X = df[feature_cols]
y = df[label_col]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42, stratify=y
)

# Perceptron pipeline
pipe = Pipeline([
  ("scaler", StandardScaler()),
  ("clf", Perceptron(max_iter=1000, tol=1e-3, class_weight="balanced", random_state=42))
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print("=== Feature used ===")
print(feature_cols)
print("\n=== Classification Report (SLP + BERT only) ===")
print(classification_report(y_test, y_pred))
print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

=== Feature used ===
['bert_similarity', 'title_length', 'sensational_word_count', 'punctuation_count', 'punctuation_ratio']

=== Classification Report (SLP + BERT only) ===
              precision    recall  f1-score   support

           0       0.88      0.35      0.50       514
           1       0.16      0.72      0.26        86

    accuracy                           0.40       600
   macro avg       0.52      0.53      0.38       600
weighted avg       0.78      0.40      0.46       600


=== Confusion Matrix ===
[[179 335]
 [ 24  62]]
